In [7]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pefile
import hashlib
import capstone
import re
import mimetypes
from tkinter.scrolledtext import ScrolledText
import threading
import os
from datetime import datetime
import getpass


def calculate_hash(file_path):
    """Calculare hash MD5."""
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


def extract_strings(file_path, min_length=4):
    """Extragere string-uri din fișier."""
    with open(file_path, "rb") as f:
        content = f.read()
    return re.findall(rb"[ -~]{%d,}" % min_length, content)


def extract_hex_dump(file_path, size=256):
    """Extragere hex dump din fișier."""
    with open(file_path, "rb") as f:
        content = f.read(size)
    return " ".join(f"{byte:02x}" for byte in content)


def get_file_metadata(file_path):
    """Extragere metadate despre fișier."""
    stat_info = os.stat(file_path)
    file_size = stat_info.st_size
    permissions = oct(stat_info.st_mode)[-3:]
    owner = getpass.getuser()
    group = "N/A"  # Grupul nu este disponibil pe Windows
    timestamps = {
        "created": datetime.fromtimestamp(stat_info.st_ctime).strftime("%Y-%m-%d %H:%M:%S"),
        "modified": datetime.fromtimestamp(stat_info.st_mtime).strftime("%Y-%m-%d %H:%M:%S"),
        "accessed": datetime.fromtimestamp(stat_info.st_atime).strftime("%Y-%m-%d %H:%M:%S"),
    }
    return file_size, permissions, owner, group, timestamps


def analyze_pe_file(file_path):
    """Analiza fișier PE pentru importuri, exporturi și cod assembly."""
    try:
        pe = pefile.PE(file_path)
        analysis_content = "PE Analysis:\n"

        # Importuri
        imports_info = "Imports:\n"
        if hasattr(pe, 'DIRECTORY_ENTRY_IMPORT'):
            for entry in pe.DIRECTORY_ENTRY_IMPORT:
                imports_info += f"\n{entry.dll.decode()}:\n"
                for imp in entry.imports:
                    imports_info += f"\t{imp.name.decode() if imp.name else 'None'}\n"
        else:
            imports_info += "Niciun import detectat.\n"

        # Exporturi
        exports_info = "Exports:\n"
        if hasattr(pe, 'DIRECTORY_ENTRY_EXPORT') and pe.DIRECTORY_ENTRY_EXPORT is not None:
            for exp in pe.DIRECTORY_ENTRY_EXPORT.symbols:
                exports_info += f"\t{exp.name.decode() if exp.name else 'None'}\n"
        else:
            exports_info += "Niciun export detectat.\n"

        # Dezasamblare cod
        assembly_info = "Assembly Code (first 20 instructions):\n"
        if hasattr(pe, 'get_memory_mapped_image'):
            code_section = pe.sections[0].get_data()
            md = capstone.Cs(capstone.CS_ARCH_X86, capstone.CS_MODE_32)
            count = 0
            for i in md.disasm(code_section[:100], pe.OPTIONAL_HEADER.ImageBase):
                if count >= 20:
                    break
                assembly_info += f"0x{i.address:x}:\t{i.mnemonic} {i.op_str}\n"
                count += 1
        else:
            assembly_info += "Nu s-a putut dezasambla codul.\n"

        return analysis_content + imports_info + "\n" + exports_info + "\n" + assembly_info
    except Exception as e:
        return f"Eroare analiză PE: {e}"


def analyze_file(file_path):
    threading.Thread(target=display_file_info, args=(file_path,), daemon=True).start()


def browse_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        analyze_file(file_path)


def get_file_hex_signature(file_path):
    """Obține semnătura hex a fișierului (primii 4 octeți)."""
    try:
        with open(file_path, "rb") as file:
            header_bytes = file.read(4)  # Citim primii 4 octeți
            hex_signature = " ".join(f"{byte:02x}" for byte in header_bytes)
            return hex_signature
    except Exception as e:
        return "Eroare citire hex"


def display_file_info(file_path):
    try:
        file_type, _ = mimetypes.guess_type(file_path)
        file_hash = calculate_hash(file_path)
        file_size, permissions, owner, group, timestamps = get_file_metadata(file_path)
        pe_analysis = analyze_pe_file(file_path)
        hex_dump = extract_hex_dump(file_path)
        strings_extracted = extract_strings(file_path)

        hex_signature = get_file_hex_signature(file_path)

        result_text.delete(1.0, tk.END)
        result_text.tag_configure("title", foreground="green", font=("Helvetica", 10, "bold"))
        result_text.tag_configure("content", foreground="blue", font=("Helvetica", 10))

        analysis_content = (
            f"File Analysis\n"
            f"File Path: {file_path}\n"
            f"File Type: {file_type or 'Unknown'} ({hex_signature})\n"
            f"File Size: {file_size} bytes\n"
            f"Permissions: {permissions}\n"
            f"Owner: {owner}\n"
            f"Group: {group}\n"
            f"Created: {timestamps['created']}\n"
            f"Modified: {timestamps['modified']}\n"
            f"Accessed: {timestamps['accessed']}\n"
            f"File Hash (MD5): {file_hash}\n\n"
            f"Hex Dump (first 256 bytes):\n{hex_dump}\n\n"
            f"{pe_analysis}\n"
            f"Extracted Strings (first 50 strings):\n"
            + "".join([s.decode(errors='ignore') + '\n' for s in strings_extracted[:50]])
        )

        # Adaug titluri și conținut cu stiluri diferite
        for line in analysis_content.split("\n"):
            if any(field in line for field in ["File Path", "File Type", "File Size", "Permissions", "Owner", "Group", "Created", "Modified", "Accessed", "File Hash"]):
                result_text.insert(tk.END, line + "\n", "title")
            else:
                result_text.insert(tk.END, line + "\n", "content")

        save_analysis(file_path, analysis_content)
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")



def save_analysis(file_path, analysis_content):
    folder_name = "Analize_Fisiere"
    os.makedirs(folder_name, exist_ok=True)
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"{folder_name}/Analiza_{timestamp}.txt"
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(analysis_content)
    messagebox.showinfo("Success", f"Analiza a fost salvată în fișierul: {file_name}")


# Interfața grafică
root = tk.Tk()
root.title("File Analyzer")

browse_button = tk.Button(root, text="Browse File", command=browse_file)
browse_button.pack(pady=10)

result_text = ScrolledText(root, width=100, height=30, wrap=tk.WORD)
result_text.pack(padx=10, pady=10)

root.mainloop()
